# 第4回 最短経路探索3

## 前回までの補足

### search contour (探索の輪郭) とは: 
  ある時点での探索中の最前線のことだが、2Dグリッドなどでは open list が作る形状として図示できる。
  前回口頭でしか説明しませんでしたので追記しておきます。
  
### タイブレーク Tiebreaking

A*探索中に $f$値が等しい節点が複数存在した場合にはどうするべきか？
とくにコスト関数が整数の場合は同じ値を持つ節点が大量に存在することが多い。
この場合にどれを優先するかを決めるのが**タイブレーキング (Tiebreaking)**である。

タイブレーキングの方法としてよく使われるのは以下の2つである。

- f値が同じなら、h値が小さい節点を優先する方法。
- オープンリストに後に入った節点を優先する LIFO (Last-in-first-out) タイブレーキング。


## メモリが限られている場合の探索 Memory Bounded Search

前回は、グラフ探索で問題を解こうとする際に探索時間が事前に分からないという問題に対して、
Anytime性を持たせる工夫を紹介した。
グラフ探索では探索時間だけではなくメモリ使用量も分からない。
今回はA*探索等で探索中に計算機のメモリが足りなくなりそうな場合、どのような方法があるのか紹介する。

### 何らかの方法で closed list を忘れる

もし、もう2度と訪問されない節点を知ることができれば
それを closed list から削除することでメモリを節約できる。
実際に、問題の性質を利用してメモリを節約できることもある。
例えば 2D grid上の最短経路探索なら4回訪問された節点は忘れて良い。
ある節点に到達する方法は4通りだけだから、もう訪問されることはないはずだ。

### ビームサーチ beam search

open list の数に上限を設けることでメモリを節約する方法である。
何らかの定数$k$を適当に決め、$f$値の良い方から$k$個だけ保存する。
当然だが以下のどちらも満たさない。

- completeness (解があるなら出力できるか)
- optimality (出力された解は最善か)

探索対象の分岐数が多すぎる場合などに用いられる。
様々なバリエーションがあり、$k$を定数とするのではなく
$f$値が最善より$d$だけ悪い節点を保存するなどの方法もある。
また、徐々に $k$ を大きくして探索を繰り返す反復広化 (Iterative Widening) を用いて
メモリが許す限り探索を行う方法もある。

最近でも自然言語の分野で文章を生成する場合などに使われており、
completeness や optimality がないからと言って実用性が無いわけではない。

### 反復深化A*探索 Iterative Deepening A* (IDA*) search

反復深化深さ優先探索 (Iterateve Deepening DFS, ID-DFS) を以前紹介したが、
それのA*探索版が IDA* 探索である。
ID-DFSではスタート節点からの距離 $g$ の上限を徐々に増やして繰り返し探索を行ったが、
IDA*では $f$ の上限を増やしながら繰り返し探索を行う。（当然 $f = g+h$ である。）

ID-DFSと同様に、同じ節点を何度も展開する性質がある。
（そのために、たとえば2D gridなどでは非効率であることが見て取れると思う。）
しかしメモリ使用量が大きくなる問題に適用した場合は
このままでも効率良く動作することがある。
ヒューリスティック関数が許容的なら (admissible heuristic を使うなら） スコアが整数で1ずつ増やしていく場合などは最善解を返す。
また、解が存在すれば発見する (complete)。
ただし、ループがあるなどの場合には節点数が有限でも停止しないことがある。

### Frontier A* search:

詳細は説明しないが、open list (= frontier) だけを覚えて探索する方法もある。
これは closed list へ戻る経路を2度とたどらないようにする方法だが、
closed list がないと言うことは探索が終了した時点で
最短経路が保存されていないということでもある。
そのために最短経路を復元するための探索がさらに必要となる。
詳細は以下などを参照。

> [Korf et al. 2005] R. E. Korf, W. Zhang, I. Thayer and H. Hohwald, "Frontier search" J. ACM, vol. 52, no. 5, pp. 715–748, 2005.

### Recursive Best First Search

これは探索中に最善の (f値が一番小さい) 経路の周辺以外は忘れていく方法である。
ただし、2番目に良い (探索中に発見された末端のf値が2番目に小さい) 経路が
どこなのか分かるように、最低限の情報だけを持っておく。
現在探索中の場所の探索が進んでf値が増大し最善ではなくなった場合、
2番目に良かったはずの経路が最善 (f値最小) となるので、その経路の探索を開始する。
これを繰り返して、最善の経路以外を忘れ、再探索することを繰り返す。

RBFSはメモリ消費量が探索深さに比例する大きさに抑えられ、
かつヒューリスティック関数が許容的なら (admissible heuristic を使うなら）
最善解を得ることができる。
ただし繰り返し同じ経路を展開するので実行時間は余分にかかる。

> [Korf 1993] R.E. Korf, "Linear-space best-first search". Artificial Intelligence. 62(1):41–78, 1993.

## メモリが限られている場合に、できるだけメモリを使う探索

上で紹介した方法は、**メモリ消費量が小さすぎる**場合がある。
直感的には、可能な範囲でメモリを使ってより高速な探索を行うことが可能なはずだ。
ここではそのような方法を紹介する。

### Simplified Memory-bounded A* (SMA*)

SMA* はメモリが余っている限り、 A* と全く同様の動作をする。
もしメモリを使い切って、closed_list や open_list に新たな節点を追加できなくなった場合、
closed_list から最悪の節点、つまり$f$値が最大の節点を一つ削除する。
このようにして削除された節点については、その節点が保持していたスコアを親節点に記録する。

ある節点$n$の子節点が全て削除されたとする。
その場合は$n$から先の経路は全く失われているが、子節点の中で最善の$f$値を残しておけば、
$n$から下を探索するべきかどうか判断することができる。
$f$値最大の節点が複数あった場合には一番古いものを忘れるようにする必要があるなど、
ちょっとした注意点を守れば最短経路を発見することができる。

以下の様な性質がある。
- 経路がメモリに載るならば発見できる (completeness)
- 最善解がメモリに載るならば発見できる (optimality)
- サイクルがあっても停止する。メモリを使い切るので。
- メモリがなくても動くが遅い。メモリがあればあるほど早い

### 遷移表つき反復深化A*探索 (IDA* with transposition table)

これは上記のIDA*探索と遷移表 (Transposition Table) を組み合わせた物だが、
遷移表とは要するにハッシュテーブルである。
動作としては単純で、$f$値の上限を適当の（小さな）値に定めて探索をはじめ、
解が見つからなければ上限を少しずつ増やすという方法である。
特に $f$値が整数でかつあまり大きく無い場合は上手く動作する。

2Dグリッドの動作例を見るとIDA*は非常に無駄なように思えるが、
ハッシュテーブルを使って過去の情報をできるだけたくさん記憶しておくことで
繰り返し探索を行う際の無駄を省くことができる。
IDA*と同様にスコアが整数で1ずつ増やしていく場合などは最善解を発見する能力があり
（注意点はあるがここでは省略する）、
メモリが十分にあればA*に近い速度で動作する。
メモリが足りない場合でも遅くなるだけで解を発見する能力は変わらない。

SMA*の場合と同様に、hash tableがメモリを使い切った場合には
何らかの基準で節点を削除していく必要がある。
適当に消してもアルゴリズムの正しさには影響しないが、
有望な節点を残す方が動作は高速になる。
今回は解説しないが、遷移表つきIDA*探索は並列化が可能である点でも重要である。

参考：
https://qiita.com/guicho271828/items/b3e885c5bde5bf7183c2

下の疑似コードは非常に単純な遷移表つきIDA*の動作を示す。
探索の最大深さを制御する max_f をゼロから始めて徐々に増やしていく。
max_f はゼロから始めているが、開始節点を$s$として、$h(s)$に初期化することはよく行われる。
また、max_f を1ずつ増やしているが、別に1ずつ増やす必要はなく、色々な工夫がありうる。
Recur関数が探索の本体で、これを再帰的に呼ぶことで探索が動く。
ハッシュテーブルがなければ（ヒューリスティック関数の有無以外は）反復深化深さ優先探索と同様の動作をするが、
もしハッシュテーブルに既に記録されていれば再探索はせずに
記録されていた結果をそのまま返す。
実際には何を記録するかバリエーションがある。
なお、疑似コード中では省略されているが、ハッシュ表に追加できなくなった場合は
その場でハッシュ表の内容を一部削除する処理を行う。


```
fun Recur(max_f, node) {
  if (goal == node) { return PATH_FOUND }

  entry <- hash_table.lookup(node)
  if (not entry.existed or entry.max_f < max_f) {
    if ( f(node) > max_f ) {
      hash_table.record( (max_f, node) )
      return NOT_FOUND
    }
    for each child in Expand(node) {
      result <- Iter(max_f, child)
      if (result == PATH_FOUND) { return result }
    }
    hash_table.record( (max_f, node) )
    return NOT_FOUND
  }
  return NOT_FOUND
}

fun IDASTar (prob, f) {
  hash_table <- start 節点のみを含む hash table
  
  max_f <- 0
  while (true) {
    result <- Recur(max_f, start_node)
    if (result == PATH_FOUND) { return result }
    max_f += 1
  }
}
```

## 第4回 課題:

### 課題の解説

15パズルの場合、単純なヒューリスティック関数の例として良く用いられるのが「ハミング距離ヒューリスティック」と言われる物である。
これは単に正解の位置にあるパネルの枚数を数えるという方法だ。
それよりはかなり良いヒューリスティック関数に「マンハッタン距離ヒューリスティック」がある。
これは、それぞれのパネルの現在位置と正解位置のマンハッタン距離を計算し、それを合計したものである。

下のコード例ではハッシュテーブルのキーとしてパズル盤面の文字列を使っているが、
これは単にコードの分かりやすさのためであって、推奨しているわけではない。
その他、分かりやすさのために速度やメモリ使用量を犠牲にしている点は多い。

課題
1. ハミング距離ヒューリスティックとマンハッタン距離ヒューリスティックはそれぞれ許容的 (admissible) か。また単調 (consistent) か。
1. 15パズルより一つ小さい8パズルを解け。以下のコードを参考にしても良い。ただし、ヒューリスティック関数してハミング距離とマンハッタン距離の2パターンを実装し、expandの回数やclosed_listの要素数を比較すること。初期盤面は適当に設定すること。  
    （ランダムに設定すると1/2の確率で解けない盤面になるので注意すること。安全のためには初期盤面から十分な回数ランダムに操作すると良い。）
1. (発展課題) IDA* 探索を実装し、8パズルを解け。さらに意欲があれば 15パズルも解け。
1. (発展課題) さらに意欲があれば遷移表つきIDA*を実装し、8パズルか15パズルを解け。その場合、ハッシュ表の上限サイズを適当に設定して人為的にメモリが足りなくなるようにした上でexpandの回数などを観察せよ。
1. (発展課題) 以下の外部リンク先の問題を解け。

### 参考 外部リンク
- 8 puzzle  
  https://judge.u-aizu.ac.jp/onlinejudge/description.jsp?id=ALDS1_13_B
- 15 puzzle  
  https://judge.u-aizu.ac.jp/onlinejudge/description.jsp?id=ALDS1_13_C

# 課題 1  
ハミング距離ヒューリスティック:    admissible, not consistent   
マンハッタン距離ヒューリスティック: admissible, consistent

# 課題 2
manhattan :  
expand: 55   
closed_list: 94
   
hamming:  
expand: 110   
closed_list: 185

# 課題 3
8-puzzle

In [1]:
from pdb import set_trace as st
import copy
import heapq

class Node:
    def __init__(self, state, g, h, parent):
        self.state = state
        self.g = g
        self.h = h
        self.parent = parent
        
    def cost(self):
        return self.g + self.h

    def __lt__(self, other):
        if (self.g + self.h) < (other.g + other.h):
            return True
        elif (self.g + self.h) == (other.g + other.h):
            return self.h < other.h
        else:
            return False

class State:
    def __init__(self, pos):
        '''
        state: panel position in 3x3 integer list
        '''
        self.pos = pos

    def __str__(self):
        string = ""
        for i in range(3):
            for j in range(3):
                string += str(self.pos[i][j])
            string += '\n'
        return string

    def __eq__(self, other):
        return self.pos == other.pos

    def get_position(self, num):
        for i in range(3):
            for j in range(3):
                if self.pos[i][j] == num:
                    return (i, j)
        # should throw exception
        return None

    def expand(self):
        '''
        return children state list
        この関数を実装せよ
        '''
        children=[]
        x,y=self.get_position(0)
        if x==0:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x+1][y]=state[x+1][y],state[x][y]
            children.append(State(state))
        if y==0:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y+1]=state[x][y+1],state[x][y]
            children.append(State(state))
        if x==1:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x+1][y]=state[x+1][y],state[x][y]
            children.append(State(state))
            state=copy.deepcopy(self.pos)
            state[x][y],state[x-1][y]=state[x-1][y],state[x][y]
            children.append(State(state))
        if y==1:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y+1]=state[x][y+1],state[x][y]
            children.append(State(state))
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y-1]=state[x][y-1],state[x][y]
            children.append(State(state))
        if x==2:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x-1][y]=state[x-1][y],state[x][y]
            children.append(State(state))
        if y==2:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y-1]=state[x][y-1],state[x][y]
            children.append(State(state))
        return children


class Puzzle:
    def __init__(self, init_pos):
        self.start = State(init_pos)
        self.goal = State([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

    def heuristic(self, state):
        '''
        ヒューリスティック関数を実装せよ
        関数名などは適当に変更してかまわない
        '''
        # manhattan
        h_value=0
        for i in range(9):
            x,y=state.get_position(i)
            x1,y1=self.goal.get_position(i)
            h_value+=abs(x-x1)+abs(y-y1)
        return h_value
                
    def hamming(self,state):
        h_value=0
        for i in range(9):
            x,y=state.get_position(i)
            x1,y1=self.goal.get_position(i)
            if x!=x1 or y!=y1:
                h_value+=1
        return h_value

class AStar:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.closed_list = { }

    def solution_path(self, goal):
        '''
        この関数はこのまま使っても良いし適当に変更しても良い
        たとえば expand の回数や closed_list の要素数などを表示するなど
        '''
        solution = []
        node = goal
        while True:
            solution = [str(node.state)] + solution
            if node.parent.pos[0][0] < 0:
                break
            node = self.closed_list[str(node.parent)]
        sol_str = "len: " + str(goal.g) + '\n'
        for pos in solution:
            sol_str += pos + '\n'
        return sol_str

    def search(self):
        '''
        以下はあくまで一例で、dummy_stateの定義などは適宜変更せよ
        '''
        start_state = self.puzzle.start
        dummy_state = State([[-1, -1, -1], [-1, -1, -1], [-1, -1, -1]])
        
        # manhattan
        start_node = Node(start_state, 0, self.puzzle.heuristic(start_state), dummy_state)
        
        # # hamming
        # start_node = Node(start_state, 0, self.puzzle.hamming(start_state), dummy_state)
        
        # open_list の初期化
        open_queue = [ start_node ]
        heapq.heapify(open_queue)
        
        # ハッシュ表の初期化。ここではハッシュ表のキーとしてStateの文字列を使っている
        self.closed_list = { }
        self.closed_list[str(start_state)] = start_node

        if start_state == self.puzzle.goal:
            return self.solution_path(start_node)
        
        count_expand=0
        while len(open_queue) != 0:
            '''
            以前の課題とほぼ同じはずだが、ここを実装せよ。
            '''
            n = heapq.heappop(open_queue)
            if n.state == self.puzzle.goal:
                print('expand:',count_expand)
                print('len_closed_list:',len(self.closed_list))
                return n
            child = n.state.expand()
            count_expand+=1
            for c in child:
                num=self.puzzle.heuristic(c)
                # num=self.puzzle.hamming(c)
                num1=n.g
                if str(c) not in self.closed_list or (num1+1+num) < self.closed_list[str(c)].cost():
                    newnode=Node(c,num1+1,self.puzzle.heuristic(c),n)
                    # newnode=Node(c,num1+1,self.puzzle.hamming(c),n)
                    self.closed_list[str(c)] = newnode
                    heapq.heappush(open_queue, newnode)
        return None
    
    def search_f_max(self,f_max,hashsize):
        start_state = self.puzzle.start
        dummy_state = State([[-1, -1, -1], [-1, -1, -1], [-1, -1, -1]])
        # manhattan
        start_node = Node(start_state, 0, self.puzzle.heuristic(start_state), dummy_state)
        
        # # hamming
        # start_node = Node(start_state, 0, self.puzzle.hamming(start_state), dummy_state)

        open_queue = [ start_node ]
        heapq.heapify(open_queue)
        self.closed_list = { }
        self.closed_list[str(start_state)] = start_node

        if start_state == self.puzzle.goal:
            return self.solution_path(start_node)
        
        count_expand=0
        while len(open_queue) != 0:
            n = heapq.heappop(open_queue)
            if n.state == self.puzzle.goal:
                print('expand:',count_expand)
                print('len_closed_list:',len(self.closed_list))
                return n
            child = n.state.expand()
            count_expand+=1
            for c in child:
                num=self.puzzle.heuristic(c)
                # num=self.puzzle.hamming(c)
                num1=n.g
                if num+num1<f_max:
                    if str(c) not in self.closed_list and len(self.closed_list)< hashsize:
                        newnode=Node(c,num1+1,self.puzzle.heuristic(c),n)
                        # newnode=Node(c,num1+1,self.puzzle.hamming(c),n)
                        self.closed_list[str(c)] = newnode
                        heapq.heappush(open_queue, newnode)
                    elif str(c) not in self.closed_list and len(self.closed_list)==hashsize:
                        max1,node1=-1,0
                        for i in self.closed_list:
                            if self.closed_list[i].cost()>max1:
                                max1=self.closed_list[i].cost()
                                node1=i
                        if (num1+1+num)<max1:
                            self.closed_list.pop(node1)
                            newnode=Node(c,num1+1,self.puzzle.heuristic(c),n)
                            # newnode=Node(c,num1+1,self.puzzle.hamming(c),n)
                            self.closed_list[str(c)] = newnode
                            heapq.heappush(open_queue, newnode)
                    elif (num1+1+num) < self.closed_list[str(c)].cost():
                        newnode=Node(c,num1+1,self.puzzle.heuristic(c),n)
                        # newnode=Node(c,num1+1,self.puzzle.hamming(c),n)
                        self.closed_list[str(c)] = newnode
                        heapq.heappush(open_queue, newnode)
        return None
        
    
    def search_transposition(self,hashsize):
        f_max=0
        while not self.search_f_max(f_max,hashsize):
            f_max+=1
        return self.search_f_max(f_max,hashsize),f_max
    
    
    

pz8 = Puzzle([[0,1,3],[4,5,6],[7,8,2]])
# pz8 = Puzzle([[1,3,0],[4,2,5],[7,8,6]])
solver = AStar(pz8)
solution = solver.search()
a8=solution
num=-1
while hasattr(a8,'parent'):
    print(str(a8.state))
    a8=a8.parent
    num+=1
print('least_num:',num)

expand: 55
len_closed_list: 94
123
456
780

123
456
708

123
406
758

103
426
758

130
426
758

136
420
758

136
402
758

136
452
708

136
452
780

136
450
782

130
456
782

103
456
782

013
456
782

least_num: 12


15-puzzle

In [42]:
from pdb import set_trace as st
import copy
import heapq

class Node:
    def __init__(self, state, g, h, parent):
        self.state = state
        self.g = g
        self.h = h
        self.parent = parent
        
    def cost(self):
        return self.g + self.h

    def __lt__(self, other):
        if (self.g + self.h) < (other.g + other.h):
            return True
        elif (self.g + self.h) == (other.g + other.h):
            return self.h < other.h
        else:
            return False

class State:
    def __init__(self, pos):
        '''
        state: panel position in 3x3 integer list
        '''
        self.pos = pos

    def __str__(self):
        string = ""
        for i in range(4):
            for j in range(4):
                string += str(self.pos[i][j]).zfill(2)+' '
            string += '\n'
        return string

    def __eq__(self, other):
        return self.pos == other.pos

    def get_position(self, num):
        for i in range(4):
            for j in range(4):
                if self.pos[i][j] == num:
                    return (i, j)
        # should throw exception
        return None

    def expand(self):
        '''
        return children state list
        この関数を実装せよ
        '''
        children=[]
        x,y=self.get_position(0)
        if x==0:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x+1][y]=state[x+1][y],state[x][y]
            children.append(State(state))
        if y==0:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y+1]=state[x][y+1],state[x][y]
            children.append(State(state))
        if x==1 or x==2:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x+1][y]=state[x+1][y],state[x][y]
            children.append(State(state))
            state=copy.deepcopy(self.pos)
            state[x][y],state[x-1][y]=state[x-1][y],state[x][y]
            children.append(State(state))
        if y==1 or y==2:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y+1]=state[x][y+1],state[x][y]
            children.append(State(state))
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y-1]=state[x][y-1],state[x][y]
            children.append(State(state))
        if x==3:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x-1][y]=state[x-1][y],state[x][y]
            children.append(State(state))
        if y==3:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y-1]=state[x][y-1],state[x][y]
            children.append(State(state))
        return children


class Puzzle:
    def __init__(self, init_pos):
        self.start = State(init_pos)
        self.goal = State([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12],[13, 14, 15, 0]])

    def heuristic(self, state):
        '''
        ヒューリスティック関数を実装せよ
        関数名などは適当に変更してかまわない
        '''
        # manhattan
        h_value=0
        for i in range(16):
            x,y=state.get_position(i)
            x1,y1=self.goal.get_position(i)
            h_value+=abs(x-x1)+abs(y-y1)
        return h_value
                
    def hamming(self,state):
        h_value=0
        for i in range(16):
            x,y=state.get_position(i)
            x1,y1=self.goal.get_position(i)
            if x!=x1 or y!=y1:
                h_value+=1
        return h_value

class AStar:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.closed_list = { }

    def solution_path(self, goal):
        '''
        この関数はこのまま使っても良いし適当に変更しても良い
        たとえば expand の回数や closed_list の要素数などを表示するなど
        '''
        solution = []
        node = goal
        while True:
            solution = [str(node.state)] + solution
            if node.parent.pos[0][0] < 0:
                break
            node = self.closed_list[str(node.parent)]
        sol_str = "len: " + str(goal.g) + '\n'
        for pos in solution:
            sol_str += pos + '\n'
        return sol_str

    def search(self):
        '''
        以下はあくまで一例で、dummy_stateの定義などは適宜変更せよ
        '''
        start_state = self.puzzle.start
        dummy_state = State([[-1, -1, -1,-1], [-1,-1, -1, -1], [-1,-1, -1, -1],[-1,-1,-1,-1]])
        
        # manhattan
        start_node = Node(start_state, 0, self.puzzle.heuristic(start_state), dummy_state)
        
        # # hamming
        # start_node = Node(start_state, 0, self.puzzle.hamming(start_state), dummy_state)
        
        # open_list の初期化
        open_queue = [ start_node ]
        heapq.heapify(open_queue)
        
        # ハッシュ表の初期化。ここではハッシュ表のキーとしてStateの文字列を使っている
        self.closed_list = { }
        self.closed_list[str(start_state)] = start_node

        if start_state == self.puzzle.goal:
            return self.solution_path(start_node)
        
        count_expand=0
        while len(open_queue) != 0:
            '''
            以前の課題とほぼ同じはずだが、ここを実装せよ。
            '''
            n = heapq.heappop(open_queue)
            if n.state == self.puzzle.goal:
                print('expand:',count_expand)
                print('len_closed_list:',len(self.closed_list))
                return n
            child = n.state.expand()
            count_expand+=1
            for c in child:
                num=self.puzzle.heuristic(c)
                # num=self.puzzle.hamming(c)
                num1=n.g
                if str(c) not in self.closed_list or (num1+1+num) < self.closed_list[str(c)].cost():
                    newnode=Node(c,num1+1,self.puzzle.heuristic(c),n)
                    # newnode=Node(c,num1+1,self.puzzle.hamming(c),n)
                    self.closed_list[str(c)] = newnode
                    heapq.heappush(open_queue, newnode)
        return None
    
    


pz15 = Puzzle([[1,3,0,4],[5,2,7,8],[9,6,10,11],[13,14,15,12]])
solver = AStar(pz15)
solution = solver.search()
a15=solution
while hasattr(a15,'parent'):
    print(str(a15.state))
    a15=a15.parent

expand: 8
len_closed_list: 21
01 02 03 04 
05 06 07 08 
09 10 11 12 
13 14 15 00 

01 02 03 04 
05 06 07 08 
09 10 11 00 
13 14 15 12 

01 02 03 04 
05 06 07 08 
09 10 00 11 
13 14 15 12 

01 02 03 04 
05 06 07 08 
09 00 10 11 
13 14 15 12 

01 02 03 04 
05 00 07 08 
09 06 10 11 
13 14 15 12 

01 00 03 04 
05 02 07 08 
09 06 10 11 
13 14 15 12 

01 03 00 04 
05 02 07 08 
09 06 10 11 
13 14 15 12 



# 課題 4

In [44]:
pz8 = Puzzle([[0,1,3],[4,5,6],[7,8,2]])
# pz8 = Puzzle([[1,3,0],[4,2,5],[7,8,6]])
solver = AStar(pz8)
solution=solver.search_transposition(100)
a8,f_max=solution
print(str(a8.state))
print('f_max:',f_max)

expand: 55
len_closed_list: 68
expand: 55
len_closed_list: 68
123
456
780

f_max: 15


In [3]:
pz8 = Puzzle([[0,1,3],[4,5,6],[7,8,2]])
# pz8 = Puzzle([[1,3,0],[4,2,5],[7,8,6]])
solver = AStar(pz8)
solution=solver.search_transposition(60)
a8,f_max=solution
if hasattr(a8,'state'):
    while hasattr(a8,'parent'):
        print(str(a8.state))
        a8=a8.parent
else:
    print('no result')

expand: 55
len_closed_list: 60
expand: 55
len_closed_list: 60
123
456
780

123
456
708

123
406
758

103
426
758

130
426
758

136
420
758

136
402
758

136
452
708

136
452
780

136
450
782

130
456
782

103
456
782

013
456
782



# 結論
too small size of hash table ----> can't get result because of loops 

# 課題 5

In [5]:
pz8 = Puzzle([[0,1,3],[4,5,6],[7,8,2]])
# pz8 = Puzzle([[1,3,0],[4,2,5],[7,8,6]])
solver = AStar(pz8)
solution = solver.search()
a8=solution
num=-1
while hasattr(a8,'parent'):
    a8=a8.parent
    num+=1
print('least_num:',num)

expand: 55
len_closed_list: 94
least_num: 12


参考文献
- "Artificial Intelligence: A Modern Approach, 4th Global ed.", by Stuart Russell and Peter Norvig  
   http://aima.cs.berkeley.edu/index.html
- "ヒューリスティック探索入門", 陣内 佑  
   https://jinnaiyuu.github.io/pdf/textbook.pdf